Influence of lambda on the result quality
=================================

In [ ]:
import sys
import os
sys.path.append(os.path.realpath("../.."))
from scripts.preamble import *
from scripts.io_ply import read_ply 

sns.set_style({'axes.grid' : False})
recompute = True
fontsize = 18

In [ ]:
basename = os.path.join(OUTPUT_DIR, os.path.basename(os.getcwd()))
alphas = pd.read_csv(os.path.join(basename, "alphas.csv"), index_col=0).iloc[:].values[:,0]
lambdas = alphas / (1-alphas)

In [ ]:
# Load wireframes
wireframes = []
for i in range(len(lambdas)):
    filename = os.path.join(basename, f"res_{i:02d}_wireframe.png")
    if not os.path.exists(filename) or recompute:
        blender_render(os.path.join(basename, f"res_{i:02d}.ply"), 
                       basename, 
                       14,
                       6,
                       wireframe=True)
    wireframes.append(plt.imread(filename))

In [ ]:
try:
    from igl import hausdorff
    hausdorff_distances = np.zeros(len(lambdas))
    mesh = read_ply(os.path.join(basename, f"ref.ply"))
    v_ref, f_ref = mesh["vertices"].cpu().numpy(), mesh["faces"].cpu().numpy()
    for i in range(len(lambdas)):
        mesh = read_ply(os.path.join(basename, f"res_{i:02d}.ply"))
        v, f = mesh["vertices"].cpu().numpy(), mesh["faces"].cpu().numpy()
        hausdorff_distances[i] = 0.5 * (hausdorff(v_ref, f_ref, v, f) + hausdorff(v, f, v_ref, f_ref))
except ModuleNotFoundError:
    print("WARNING: could not import libigl. The Hausdorff distances will not be computed. Please install libigl if you want to compute them.")
    hausdorff_distances = None

In [ ]:
base_size = 3
flat = False
h_dist = True
if flat:
    n_rows = 1
    n_cols = diffs.shape[0]
else:
    n_cols = 4
    n_rows = len(wireframes) // n_cols
H,W,_ = wireframes[0].shape
w = int(0.84*H)
aspect = w/H
fontsize = 24

fig = plt.figure(1, figsize=(n_cols * base_size* aspect, n_rows * base_size*1.3), constrained_layout=True)
gs = fig.add_gridspec(n_rows,  n_cols, wspace=0.075,hspace=0.0)

for i in range(n_rows):
    for j in range(n_cols):
        ax = fig.add_subplot(gs[i,j])
        ax.tick_params(
            axis='both',
            which='both',
            labelleft=False,
            labelbottom=False)
        
        idx = i*n_cols+j
        rnd = wireframes[idx].copy()
        rnd[rnd[..., -1] == 0] = 1
        rnd = rnd*rnd[..., -1][..., None] + (1-rnd[..., -1][..., None]) # Alpha
        rnd = rnd[..., :3]
        
        im = ax.imshow(rnd[:, (W-w)//2:(W+w)//2],)
        if hausdorff_distances is not None:
            ax.set_xlabel(f"$H$={hausdorff_distances[idx]:.3e}", fontsize=2*fontsize//3)
        if flat:
            ax.set_title(fr"${lambdas[idx]:.1f}$", y=-0.3, fontsize=fontsize)
        else:
            ax.set_title(fr"${lambdas[idx]:.1f}$", y=-0.3, fontsize=fontsize)

if flat:
    arrow = matplotlib.patches.FancyArrowPatch(
    (0.05,-0.05), (0.95,-0.05), transform=fig.transFigure,fc='black', mutation_scale = 40.)
    fig.patches.append(arrow)
    fig.suptitle(r"$\lambda$", y=-0.1, fontsize=fontsize)
else:
    arrow = matplotlib.patches.FancyArrowPatch(
    (0.05,0.48), (0.95,0.48), transform=fig.transFigure,fc='black', mutation_scale = 40.)
    fig.patches.append(arrow)
    plt.figtext(0.5,0.45, r"$\lambda$", ha="center", va="top", fontsize=fontsize)
    arrow = matplotlib.patches.FancyArrowPatch(
    (0.05,-0.03), (0.95,-0.03), transform=fig.transFigure,fc='black', mutation_scale = 40.)
    fig.patches.append(arrow)
    fig.suptitle(r"$\lambda$", y=-0.06, fontsize=fontsize)

plt.savefig(os.path.join(basename, "influence.pdf"), format='pdf', dpi=300, bbox_inches='tight', pad_inches=0.005)